In [6]:
from src import utils

In [5]:
#import required libraries
import geopandas as gpd
import xarray as xr
import pandas as pd
import os, glob
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.colors
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.cm as cm

import rioxarray as rio
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.windows import Window, transform as rio_window_transform
from rasterio.windows import from_bounds
from rasterio.transform import Affine
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.features import rasterize
from shapely.geometry import box

import dask
import dask.array as da
from dask.delayed import delayed

from osgeo import gdal, ogr, osr

import seaborn as sns
import zipfile

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\4\ipykernel_1206344\3466570486.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (http

In [ ]:
from src import utils
import importlib
importlib.reload(utils)
#from src.utils import *

print(dir(utils))
print(utils.zip_files_without_structure)

0. Load EFFIS data

In [6]:
# Path to data folders
project_f = r'L:\f02_data\wildfires\spatial_data'

# Create output folder structure
outfolder =  os.path.join(project_f, 'output')
os.makedirs(outfolder, exist_ok=True)

outFIREID_folder =  os.path.join(project_f, 'output', 'FIRE_ID')
os.makedirs(outFIREID_folder, exist_ok=True)

outFIREYEARDOY_folder =  os.path.join(project_f, 'output', 'FIRE_YEARDOY')
os.makedirs(outFIREYEARDOY_folder, exist_ok=True)

In [7]:
# load wildfires data
effis_path =  os.path.join(project_f, 'input', 'effis_rawdata_20240402', 'modis.ba.poly.shp')
effis = gpd.read_file(effis_path)

# read effis id as an integer
effis['id'] = effis['id'].astype(int)

# Convert the column containing dates to datetime format
effis['date'] = pd.to_datetime(effis['FIREDATE'])

# Change the CRS to EPSG:3035
effis = effis.to_crs(epsg=3035)

# Verify the new CRS
print("New CRS:", effis.crs)

New CRS: EPSG:3035


In [ ]:
effis.head()

In [ ]:
# create a lighter subset with target columns
effis_sel = effis[['id', 'COUNTRY',
 'FIREDATE','Year',
 'Month','date','AREA_HA','geometry']]

In [ ]:
#create additional fields to include DOY
effis_sel['doy'] = effis_sel['date'].dt.dayofyear
year2dig = (effis_sel.Year.astype(str).str[-2:])
year000 = year2dig+str('000')
effis_sel['yeardoy'] =year000.astype(int)+ effis_sel['doy'] 
effis_sel.tail()

1. Define reference raster to copy parameters

In [ ]:
# Reference raster
refrast_path = r'M:\f01_dims\ADMbndEEA39v2021\f02_upload_to_jedi\Adm_EEA38_NUTS2021v2022.tif'

# Open the raster data and taking extent and resolution parameters
refraster = rasterio.open(refrast_path)
raster_width = refraster.width
raster_height = refraster.height
minx = refraster.bounds.left
maxy = refraster.bounds.top
resolution = 100

# create list for analysed years
yr_ls = effis_sel.Year.unique().tolist()

2. Rasterize FIREID for each year

In [ ]:
# Loop through years
for yr in yr_ls:    
    effis_yr = effis_sel.query(f'Year == {yr}')
    print(f'year {yr} - nr of fires {effis_yr.id.max()}')
    
    # Start creating one raster per each year using the FIREID
    outfilename = f'FIREID_{yr}.tif'
    outfile = os.path.join(outFIREID_folder, outfilename) 
    if not os.path.exists(outfile):
        rasterize_vector_to_raster(
        effis_yr, 
        'id',
        outfile, 
        raster_width, 
        raster_height, 
        minx, 
        maxy, 
        resolution=resolution, 
        raster_crs='EPSG:3035', 
        chunk_size=1000, 
        compression='lzw'
    )

In [ ]:
from src.utils import *
from src import utils

# zip folder content and save in the same folder
folder_to_zip = outFIREID_folder
output_zip_file = os.path.join(outFIREID_folder, 'FIREID_0023.zip')

utils.zip_files_without_structure(folder_to_zip, output_zip_file)

In [ ]:
print(os.path.join(project_f, 'output', 'FIRE_ID', 'FIREID_0023.zip'))

#### Create LUT

In [12]:
effis22 = effis.query('Year==2022')

In [13]:
effis22.query('id==59324')

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,...,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,Year,Month,geometry,date
46689,59324,2022-01-24 13:10:00,2022-01-24 13:10:00,FR,Pyrénées-Atlantiques,Larrau,325,10.479041916164528,0,0,...,85.02994011973503,2.3952095808376064,0,0,100,FireSeason,2022,1,"POLYGON ((3431194.276 2278228.189, 3431214.924...",2022-01-24 13:10:00


In [14]:
effis22.id.describe()

count     13158.000000
mean     192044.918909
std       28419.860127
min       58630.000000
25%      183871.250000
50%      199786.500000
75%      205711.500000
max      211851.000000
Name: id, dtype: float64

In [8]:
effis23 = effis.query('Year==2023')

In [9]:
effis23.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['id', 'FIREDATE', 'LASTUPDATE', 'COUNTRY', 'PROVINCE', 'COMMUNE',
       'AREA_HA', 'BROADLEA', 'CONIFER', 'MIXED', 'SCLEROPH', 'TRANSIT',
       'OTHERNATLC', 'AGRIAREAS', 'ARTIFSURF', 'OTHERLC', 'PERCNA2K', 'CLASS',
       'Year', 'Month', 'geometry', 'date'],
      dtype='object')>

In [10]:
effis23=effis23[['id', 'FIREDATE', 'LASTUPDATE', 'COUNTRY', 'PROVINCE', 'COMMUNE',
       'AREA_HA', 'Year', 'Month', 'geometry', 'date']]

In [11]:
effis23.id.describe()

count      9376.000000
mean     218147.977709
std        3736.391657
min      211820.000000
25%      214746.500000
50%      218330.500000
75%      221511.250000
max      225583.000000
Name: id, dtype: float64

3. Rasterize FIRE YEARDOY for each year

In [ ]:
# Loop through years
for yr in yr_ls:    
    effis_yr = effis_sel.query(f'Year == {yr}')
    
    # Start creating one raster per each year using the FIREID
    outfilename = f'FIRE_YEARDOY_{yr}.tif'
    outfile = os.path.join(outFIREYEARDOY_folder, outfilename) 
    if not os.path.exists(outfile):
        rasterize_vector_to_raster(
        effis_yr, 
        'yeardoy',
        outfile, 
        raster_width, 
        raster_height, 
        minx, 
        maxy, 
        resolution=resolution, 
        raster_crs='EPSG:3035', 
        chunk_size=1000, 
        compression='lzw'
    )

In [ ]:
# zip folder content and save in the same folder
folder_to_zip = outFIREID_folder
output_zip_file = os.path.join(outFIREYEARDOY_folder, 'FIREYEARDOY_0023.zip')

utils.zip_files_without_structure(folder_to_zip, output_zip_file)